#### ALGORITMO DE SUSTRACCION DE FONDO

In [308]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
%matplotlib inline
#%matplotlib

# OpenCV-Python utiliza NumPy para el manejo de imágenes
import numpy as np
# cv2 es el módulo python para acceder a OpenCV 
import cv2 as cv
# Usamos las poderosas herramientas de graficación de matplotlib para mostrar imágenes, perfiles, histogramas, etc
import matplotlib.pyplot as plt

#### Funcion de sustraccion de fondo

In [309]:
def backgruond_suppress(filename, n_frame_back):
    
    # Leemos el video y el primer frame para obtener el tamaño del mismo
    capture = cv.VideoCapture(filename)
    ret, frame = capture.read(0)
    w, h, d = frame.shape

    nframe = 0
    frame_back = 0

    # Cantidad total de frames que contiene el video
    total_framecount= int(capture.get(cv.CAP_PROP_FRAME_COUNT))

    # De la cantidad total, seleccionamos de manera random los n_frame_back utilizados para generar el fondo
    random_frames = np.random.choice(total_framecount, n_frame_back)

    # Generamos una matriz de ceros para cada canal
    background_frames_b = np.zeros([w,h,n_frame_back])
    background_frames_g = np.zeros([w,h,n_frame_back])
    background_frames_r = np.zeros([w,h,n_frame_back])

    while True:
    
        ret, frame = capture.read()

        if frame is None:
            break

        nframe += 1

        if nframe in random_frames:

            back_b, back_g, back_r = cv.split(frame)

            background_frames_b[:,:,frame_back] = back_b
            background_frames_g[:,:,frame_back] = back_g
            background_frames_r[:,:,frame_back] = back_r

            frame_back += 1
    
    # Calculamos la mediana para cada canal
    median_b = np.median(background_frames_b, axis=2)
    median_g = np.median(background_frames_g, axis=2)
    median_r = np.median(background_frames_r, axis=2)

    # Hacemos un merge de los canales para obtener finalmente el fondo
    backgroung = cv.merge((median_b,median_g,median_r))
    backgroung = backgroung.astype(np.uint8)

    return backgroung

#### Funcion de generacion de la mascara para cada frame

In [349]:
def main_sustrator(filename, n_frame_back, refresh):

    capture = cv.VideoCapture(filename)

    n_act = 0

    if not capture.isOpened:
        print('Falla al abrir el archivo: ' + filename)
        exit(0)

    while True:
        
        # Leemos un frame
        ret, frame = capture.read()
        if frame is None:
            break
        
        # calculamos los seg del video
        video_sec = capture.get(cv.CAP_PROP_POS_MSEC)/1000
        #print(video_sec)

        if video_sec >= refresh * n_act:
            
            n_act += 1
            # Generamos el background
            print('Actualizamos el fondo')
            background = backgruond_suppress(filename, n_frame_back)
    
        else:
                 
            # Aplicamos la sustracción al frame leído
            background_gray = cv.cvtColor(background, cv.COLOR_BGR2GRAY)
            frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

            # COnvertimos a float para poder restar. Los valores dentro de un umbral alrededor de cero los ponemos en cero y volvemos a convertir a unit8
            background_gray = background_gray.astype(float)
            frame_gray = frame_gray.astype(float)
            fgMask0 =  frame_gray - background_gray
            fgMask0[(-15 < fgMask0) & (fgMask0 < 15)] = 0
            fgMask = fgMask0.astype('uint8')

            # Binarizamos
            ret, thresh = cv.threshold(fgMask,25,255,cv.THRESH_BINARY)
            
            # Creamos un elemento estructurante y aplicamos operaciones morfologicas
            kernel = np.ones((3,3),np.uint8)
            opening = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel, iterations = 1)
            #opening = thresh

            # Escribimos sobre la imagen el número de frame procesado
            cv.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
            cv.putText(frame, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
                    cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
            
            # mostramos frame original e imagen binaria background/foreground
            cv.imshow('Frame', frame)
            cv.imshow('FG Mask', opening)
            
            # Corresmos hasta que termine o apriete escape
            keyboard = cv.waitKey(30)
            if keyboard == 'q' or keyboard == 27:
                break   
             
    cv.destroyAllWindows()
    capture.release()

In [352]:
filename = 'C:/Users\juana\OneDrive\Documentos\IA\CEIA\VPC1\Codigo\Clase8/vtest.avi'

# main_sustrator(filename,  n_frame_back, refresh), donde:
# - filename = nombre del archivo de video
# - n_frame_back = cantidad de frame para obtener el fondo
# - refresh = tiempo en seg de actualizacion del fondo
main_sustrator(filename, 50, 10)


Actualizamos el fondo
Actualizamos el fondo
Actualizamos el fondo
Actualizamos el fondo
Actualizamos el fondo
Actualizamos el fondo
Actualizamos el fondo
Actualizamos el fondo


In [353]:
filename = 'C:/Users\juana\OneDrive\Documentos\IA\CEIA\VPC1\Codigo\Clase8/slow_traffic_small.mp4'

# main_sustrator(filename,  n_frame_back, refresh), donde:
# - filename = nombre del archivo de video
# - n_frame_back = cantidad de frame para obtener el fondo
# - refresh = tiempo en seg de actualizacion del fondo
main_sustrator(filename, 25, 10)

Actualizamos el fondo
Actualizamos el fondo
Actualizamos el fondo
Actualizamos el fondo


#### Comparacion con los algoritmos de mezclas de Gaussianas

In [ ]:
metodo = 'MOG2'
#metodo = 'KNN'
if metodo == 'MOG2':
    backSub = cv.createBackgroundSubtractorMOG2()
else:
    backSub = cv.createBackgroundSubtractorKNN()
    backSub = cv.createBackgroundSubtractorKNN(detectShadows = True)

# Abrimos el archivo
#-------------------
#filename = 'C:/Users\juana\OneDrive\Documentos\IA\CEIA\VPC1\Codigo\Clase8/vtest.avi'
filename = 'C:/Users\juana\OneDrive\Documentos\IA\CEIA\VPC1\Codigo\Clase8/slow_traffic_small.mp4'
capture = cv.VideoCapture(filename)

if not capture.isOpened:
    print('Falla al abrir el archivo: ' + filename)
    exit(0)

# Corremos la sustraccion
#------------------------
while True:
    # Leemos un frame
    ret, frame = capture.read()
    if frame is None:
        break
    
    # Aplicamos la sustracción al frame leído
    #----------------------------------------
    # Cada frame se utiliza tanto para calcular la máscara de primer plano como para actualizar el fondo.
    # Si se desea cambiar la tasa de aprendizaje utilizada para actualizar el modelo de fondo, es posible
    # establecer una tasa de aprendizaje específica pasando un parámetro al método apply.
    fgMask = backSub.apply(frame)
    
    # Escribimos sobre la imagen el número de frame procesado
    cv.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(frame, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
    
    # mostramos frame original e imagen binaria background/foreground
    cv.imshow('Frame', frame)
    cv.imshow('FG Mask', fgMask)
    
    # Corresmos hasta que termine o apriete escape
    keyboard = cv.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break

cv.destroyAllWindows()
capture.release()

Con el algoritmo implementado de naive para detectar el fondo usando la mediana como estimador se consiguen resultados similares a los de Opencv de mezclas de Gaussianas, considerando como aceptable a este para la deteccion de objetos en movimiento. Paca cada tipo de video, es posible ajustar algunos valores que estan dentro de la funcion, de manera de hacer mejor la deteccion.